## Extract Poses from Amass Dataset

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook
%matplotlib inline

import sys, os
import torch
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm
import rotation_conversions as rc


from human_body_prior.tools.omni_tools import copy2cpu as c2c

os.environ['PYOPENGL_PLATFORM'] = 'egl'

### Please remember to download the following subdataset from AMASS website: https://amass.is.tue.mpg.de/download.php. Note only download the <u>SMPL+H G</u> data.
* ACCD (ACCD)
* HDM05 (MPI_HDM05)
* TCDHands (TCD_handMocap)
* SFU (SFU)
* BMLmovi (BMLmovi)
* CMU (CMU)
* Mosh (MPI_mosh)
* EKUT (EKUT)
* KIT  (KIT)
* Eyes_Janpan_Dataset (Eyes_Janpan_Dataset)
* BMLhandball (BMLhandball)
* Transitions (Transitions_mocap)
* PosePrior (MPI_Limits)
* HumanEva (HumanEva)
* SSM (SSM_synced)
* DFaust (DFaust_67)
* TotalCapture (TotalCapture)
* BMLrub (BioMotionLab_NTroje)

### Unzip all datasets. In the bracket we give the name of the unzipped file folder. Please correct yours to the given names if they are not the same.

### Place all files under the directory **./amass_data/**. The directory structure shoud look like the following:  
./amass_data/  
./amass_data/ACCAD/  
./amass_data/BioMotionLab_NTroje/  
./amass_data/BMLhandball/  
./amass_data/BMLmovi/   
./amass_data/CMU/  
./amass_data/DFaust_67/  
./amass_data/EKUT/  
./amass_data/Eyes_Japan_Dataset/  
./amass_data/HumanEva/  
./amass_data/KIT/  
./amass_data/MPI_HDM05/  
./amass_data/MPI_Limits/  
./amass_data/MPI_mosh/  
./amass_data/SFU/  
./amass_data/SSM_synced/  
./amass_data/TCD_handMocap/  
./amass_data/TotalCapture/  
./amass_data/Transitions_mocap/  

**Please make sure the file path are correct, otherwise it can not succeed.**

In [ ]:
# Choose the device to run the body model on.
comp_device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
!ln -s ../datasets/AMASS_SMPLX ./amass_data

In [ ]:
import smplx
smplx_bm = smplx.create(
            "../datasets/hub/smplx_models/", 
            model_type='smplx',
            gender='NEUTRAL_2020', 
            use_face_contour=False,
            num_betas=300,
            num_expression_coeffs=100, 
            ext='npz',
            use_pca=False,
        ).cuda().eval()

def load_amass(data):
    ## 这个是用来
    # 修改amass数据里面的朝向，原本在blender里面是Z轴向上，目标是Y轴向上，当时面向目前没改
    
    data_dict = {key: data[key] for key in data}
    frames = data_dict['poses'].shape[0]
    # b = data_dict['poses'][...,:3]
    # b = rc.axis_angle_to_matrix(torch.from_numpy(b))
    # rot_matrix = np.array([[1.0, 0.0, 0.0], [0.0 , 0.0, 1.0], [0.0, -1.0, 0.0]])
    # c = np.einsum('ij,kjl->kil',rot_matrix,b)
    # c = rc.matrix_to_axis_angle(torch.from_numpy(c))
    # data_dict['poses'][...,:3] = c
    
    # trans_matrix1 = np.array([[1.0, 0.0, 0.0], [0.0 , 0.0, -1.0], [0.0, 1.0, 0.0]])
    # data_dict['trans'] = np.einsum("bi,ij->bj",data_dict['trans'],trans_matrix1)
    
    betas300 = np.zeros(300)
    betas300[:16] = data_dict['betas']
    data_dict['betas'] = betas300
    data_dict["expressions"] = np.zeros((frames,100))
    return data_dict



In [ ]:
paths = []
folders = []
dataset_names = []
for root, dirs, files in os.walk('./amass_data'):
#     print(root, dirs, files)
#     for folder in dirs:
#         folders.append(os.path.join(root, folder))
    folders.append(root)
    for name in files:
        dataset_name = root.split('/')[2]
        if dataset_name not in dataset_names:
            dataset_names.append(dataset_name)
        paths.append(os.path.join(root, name))

In [ ]:
save_root = './pose_data'
save_folders = [folder.replace('./amass_data', './pose_data') for folder in folders]
for folder in save_folders:
    os.makedirs(folder, exist_ok=True)
group_path = [[path for path in paths if name in path] for name in dataset_names]
print(group_path)

In [ ]:
trans_matrix = np.array([[1.0, 0.0, 0.0],
                            [0.0, 0.0, 1.0],
                            [0.0, 1.0, 0.0]])
ex_fps = 30
target_joint = list(range(22))+list(range(25,55))
def amass_to_pose(src_path, save_path):
    bdata = np.load(src_path, allow_pickle=True)
    if len(bdata.files)==6:
        #print(f"# ---- state file ---- #")
        return 0
    bdata = load_amass(bdata)
    fps = 0
    try:
        fps = bdata['mocap_frame_rate']
        frame_number = bdata['trans'].shape[0]
    except:
#         print(list(bdata.keys()))
        return fps
    fId = 0 # frame id of the mocap sequence
    pose_seq = []    
    
    betas = bdata['betas'].reshape(1,300)
    betas = np.tile(betas, (frame_number, 1))
    poses = bdata['poses']
    trans = bdata['trans']
    exps = bdata['expressions']
    

     
    down_sample = int(fps / ex_fps)
#     print(frame_number)
#     print(fps)
    betas = torch.from_numpy(betas).float().to(comp_device)[::down_sample]
    poses = torch.from_numpy(poses).float().to(comp_device)[::down_sample]
    trans = torch.from_numpy(trans).float().to(comp_device)[::down_sample]
    exps = torch.from_numpy(exps).float().to(comp_device)[::down_sample]
    
    
    n, c = poses.shape[0], poses.shape[1]
    max_length = 128
    s, r = n//max_length, n%max_length
    #print(n, s, r)
    all_tensor = []
    with torch.no_grad():
        for i in range(s):
            with torch.no_grad():
                joints = smplx_bm(
                    betas=betas[i*max_length:(i+1)*max_length], 
                    transl=trans[i*max_length:(i+1)*max_length], 
                    expression=exps[i*max_length:(i+1)*max_length], 
                    jaw_pose=poses[i*max_length:(i+1)*max_length, 66:69], 
                    global_orient=poses[i*max_length:(i+1)*max_length,:3], 
                    body_pose=poses[i*max_length:(i+1)*max_length,3:21*3+3], 
                    left_hand_pose=poses[i*max_length:(i+1)*max_length,25*3:40*3], 
                    right_hand_pose=poses[i*max_length:(i+1)*max_length,40*3:55*3], 
                    return_verts=True,
                    return_joints=True,
                    leye_pose=poses[i*max_length:(i+1)*max_length, 69:72], 
                    reye_pose=poses[i*max_length:(i+1)*max_length, 72:75],
                )['joints'][:, target_joint, :]
            pose_seq.append(joints)
        if r != 0:
            with torch.no_grad():
                joints = smplx_bm(
                    betas=betas[s*max_length:s*max_length+r], 
                    transl=trans[s*max_length:s*max_length+r], 
                    expression=exps[s*max_length:s*max_length+r], 
                    jaw_pose=poses[s*max_length:s*max_length+r, 66:69], 
                    global_orient=poses[s*max_length:s*max_length+r,:3], 
                    body_pose=poses[s*max_length:s*max_length+r,3:21*3+3], 
                    left_hand_pose=poses[s*max_length:s*max_length+r,25*3:40*3], 
                    right_hand_pose=poses[s*max_length:s*max_length+r,40*3:55*3], 
                    return_verts=True,
                    return_joints=True,
                    leye_pose=poses[s*max_length:s*max_length+r, 69:72], 
                    reye_pose=poses[s*max_length:s*max_length+r, 72:75],
                )['joints'][:, target_joint, :]
            pose_seq.append(joints)

    pose_seq = torch.cat(pose_seq, dim=0)
    
    pose_seq_np = pose_seq.detach().cpu().numpy()
    pose_seq_np_n = np.dot(pose_seq_np, trans_matrix)
    
    np.save(save_path, pose_seq_np_n)
    return fps

In [ ]:
group_path = group_path
all_count = sum([len(paths) for paths in group_path])
cur_count = 0
print(group_path)

This will take a few hours for all datasets, here we take one dataset as an example

To accelerate the process, you could run multiple scripts like this at one time.

In [ ]:
for paths in group_path:
    dataset_name = paths[0].split('/')[2]
    if 'BMLhandball' not in dataset_name or 'DFaust_67' not in dataset_name or 'HumanEva' not in dataset_name : continue
    print(1)

In [ ]:
import time
for paths in group_path:
    dataset_name = paths[0].split('/')[2]
    pbar = tqdm(paths)
    pbar.set_description('Processing: %s'%dataset_name)
    fps = 0
    for path in pbar:
        save_path = path.replace('./amass_data', './pose_data1')
        save_path = save_path[:-3] + 'npy'
        fps = amass_to_pose(path, save_path)
        
    cur_count += len(paths)
    print('Processed / All (fps %d): %d/%d'% (fps, cur_count, all_count) )
    time.sleep(0.5)

In [ ]:
a = np.load('')

The above code will extract poses from **AMASS** dataset, and put them under directory **"./pose_data"**

The source data from **HumanAct12** is already included in **"./pose_data"** in this repository. You need to **unzip** it right in this folder.

## Segment, Mirror and Relocate Motions

In [ ]:
import codecs as cs
import pandas as pd
import numpy as np
from tqdm import tqdm
from os.path import join as pjoin

In [ ]:
def swap_left_right(data):
    assert len(data.shape) == 3 and data.shape[-1] == 3
    data = data.copy()
    data[..., 0] *= -1
    right_chain = [2, 5, 8, 11, 14, 17, 19, 21]
    left_chain = [1, 4, 7, 10, 13, 16, 18, 20]
    left_hand_chain = [22, 23, 24, 34, 35, 36, 25, 26, 27, 31, 32, 33, 28, 29, 30]
    right_hand_chain = [43, 44, 45, 46, 47, 48, 40, 41, 42, 37, 38, 39, 49, 50, 51]
    tmp = data[:, right_chain]
    data[:, right_chain] = data[:, left_chain]
    data[:, left_chain] = tmp
    if data.shape[1] > 24:
        tmp = data[:, right_hand_chain]
        data[:, right_hand_chain] = data[:, left_hand_chain]
        data[:, left_hand_chain] = tmp
    return data

In [ ]:
index_path = '../index.csv'
save_dir = './joints'
index_file = pd.read_csv(index_path)
total_amount = index_file.shape[0]
fps = 30

In [ ]:
mis_count = 0
for i in tqdm(range(total_amount)):
    source_path = index_file.loc[i]['source_path']
    new_name = index_file.loc[i]['new_name']
    if not os.path.exists(source_path):
        # print(f"{source_path} not exists")
        mis_count = mis_count+1
        continue
    data = np.load(source_path)
    start_frame = int(index_file.loc[i]['start_frame']/20*fps)
    end_frame = int(index_file.loc[i]['end_frame']/20*fps)
    if 'humanact12' not in source_path:
        if 'Eyes_Japan_Dataset' in source_path:
            data = data[3*fps:]
        if 'MPI_HDM05' in source_path:
            data = data[3*fps:]
        if 'TotalCapture' in source_path:
            data = data[1*fps:]
        if 'MPI_Limits' in source_path:
            data = data[1*fps:]
        if 'Transitions_mocap' in source_path:
            data = data[int(0.5*fps):]
        data = data[start_frame:end_frame]
        data[..., 0] *= -1
    
    data_m = swap_left_right(data)
#     save_path = pjoin(save_dir, )
    np.save(pjoin(save_dir, new_name), data)
    np.save(pjoin(save_dir, 'M'+new_name), data_m)
print(mis_count)